# A bunch of paintings

In this notebook, I'll just make a bunch of paintings using photos I have on hand. I wanna wrap my head around all the different parameters of the model.

In [1]:
# Set working directory
%cd ..

/home/jovyan/stylized-neural-painting


In [2]:
# Import libraries
import os
import torch
from itertools import product
from attrdict import AttrDict
from painter import ProgressivePainter
from demo_prog import optimize_x
from IPython.display import display, Video

# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Get the images

Here, we list all of the images we are going to paint

In [3]:
image_dir = './test_images'
images = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
print(f'There are {len(images)} images in ./{image_dir}/')

There are 28 images in ././test_images/


## Download the renderers

We need to make sure we have all of the required renders on disk. Fortunately, the makefile is set up to grab those for us.

In [4]:
! make all -j

make: Nothing to be done for 'all'.


## Run the painters

Now we run the painters across all of the images in the `test_images` folder.

In [5]:
# Delete all the outputs
! rm -rf output/*

In [6]:
image_dir = './test_images'
images = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]

In [ ]:
# Set parameters
pic_name = 'apple'
renderers = ['oilpaintbrush', 'markerpen', 'rectangle', 'watercolor']
for image, renderer in product(images, renderers):
    image_name = image.rsplit('/', 1)[-1].rsplit('.', 1)[0]
    print(f'Image={image}, Renderer={renderer}')
    painter_args = AttrDict(
        img_path                = image,
        renderer                = renderer,
        canvas_color            = 'white',
        canvas_size             = 512,
        keep_aspect_ratio       = True,
        max_m_strokes           = 2000,
        max_divide              = 5,
        beta_L1                 = 1.0,
        with_ot_loss            = False,
        beta_ot                 = 0.1,
        net_G                   = 'zou-fusion-net',
        renderer_checkpoint_dir = f'./checkpoints_G_{renderer}',
        lr                      = 2e-4,
        output_dir              = f'./output/{image_name}_{renderer}',
        disable_preview         = True
    )

    # Initialise a progressive painter
    pt = ProgressivePainter(args=painter_args)

    # Paint the image
    optimize_x(pt)
    
    # Display the video
    display(Video(f"output/{image_name}_{renderer}/{image_name}.mp4", embed=True))

Image=./test_images/apple.jpg, Renderer=oilpaintbrush
iteration step 467, G_loss: 0.03949, step_acc: 24.39416, grid_scale: 5 / 5, strokes: 36 / 36
saving stroke parameters...
Saving video file to: ./output/apple_oilpaintbrush/apple_animated.mp4
saving input photo...
saving final rendered result...


Image=./test_images/apple.jpg, Renderer=markerpen
iteration step 451, G_loss: 0.06519, step_acc: 19.79929, grid_scale: 3 / 5, strokes: 35 / 36